# Homework 3 - What is the best anime in the world?

- Martina Milazzo
- Dimitri Saliola
- Roberta Giorgi




## 1. Data collection

In [ ]:
import requests as r
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
from datetime import datetime
import time
import csv
import numpy as np
import pandas as pd

### 1.1. Get the list of animes

In [ ]:
#383 pages * 50 animes (19130 total)

with open("C:/Users/marti/Desktop/HW3/list_anime.txt", "w", encoding='utf-8') as file:
    
    for page in tqdm(range(0, 383)):
    
        url = 'https://myanimelist.net/topanime.php?limit=' + str(page*50) #single page URL
        response = r.get(url)
        soup = BeautifulSoup(response.text, 'html.parser') #get html

        #extract all the links
        for tag in soup.find_all("tr", class_="ranking-list"):
            a_list = tag.find_all('a', class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True)
            for a in a_list:
                link = a['href']
                file.write(str(link) + '\n')
                
file.close()

100%|██████████| 383/383 [05:05<00:00,  1.25it/s]


### 1.2. Crawl animes

In [ ]:
#EXECUTE THIS CELL JUST ONES#

file = open("C:/Users/marti/Desktop/HW3/list_anime.txt", "r", encoding='utf-8')

pages_dir="C:/Users/marti/Desktop/HW3/Pages"

if not os.path.exists(pages_dir):
    os.makedirs(pages_dir)

counter_anime = 855
page = 18

for line in file:

    counter_anime += 1
    if (counter_anime%50 == 1):
        page +=1

    request = r.get(line, headers={'Cache-Control': 'no-cache'})
    if str(request.status_code)[0] == '4': 
        while(True): 
            time.sleep(120)
            request = r.get(line, headers={'Cache-Control': 'no-cache'})
            print(request.status_code)
            if str(request.status_code)[0] == '2': 
                break       
            

    soup = BeautifulSoup(request.text, 'html.parser')

    folder_dir=pages_dir+"/Page"+str(page)
    if not os.path.exists(folder_dir):
        os.makedirs(folder_dir)

    txt_dir=folder_dir + "/article_" + str(counter_anime)+ ".html"
    
    with open(txt_dir, "w", encoding='utf-8') as file:
        file.write(str(soup))

In [13]:
# Count anime links in file

file = open("C:\\Users\\marti\\Desktop\\HW3\\list_anime.txt", "r", encoding='utf-8')
line_count = 0
for line in file:
    if line != "\n":
        line_count += 1
file.close()

print(line_count)

19131


### 1.3 Parse downloaded pages

In [ ]:
#EXECUTE THIS CELL JUST ONES#

title = []
types = []
numEpisode = []
release = []
end = []
numMembers = []
score = []
users = []
rank = []
popularity = []
description = []
related = []
characters = []
voices = []
staff = []

pages = os.listdir("C:/Users/marti/Desktop/HW3/Pages")[1:]

for p in pages:
    n_html = os.listdir(pages+"/"+str(p))
    for i in range(1, len(n_html)+1):
        file = open(pages+"/"+str(p)+"/article_"+str(i), 'r', encoding="utf8")
        anime = BeautifulSoup(file, 'lxml')
        
        #TITLE
        try:
            t=anime.strong.contents[0]
            title.append(t)
        except:
            title.append('NA')
            

        #TYPE
        try:
            ty=anime.find(text = 'Type:').find_next('a').contents[0]
            types.append(ty)
        except:
            types.append('NA')


        #NUM_EPISODE
        try:
            n=int(anime.find(text = 'Episodes:').next_element.strip())
            numEpisode.append(n)
        except:
            numEpisode.append(n)


        #RELEASE_DATE
        try:
            date = anime.find(text = 'Aired:').next_element.strip()
            if len(date)>15:
                rl=datetime.strptime(date.split(" to ")[0], '%b %d, %Y' )
            else:
                rl=datetime.strptime(date, '%b %d, %Y' )
            release.append(rl)
        except:
            release.append('NA')


        #END_DATE
        try:
            date = anime.find(text = 'Aired:').next_element.strip()
            if len(date)>15 and date.split(" to ")[1] != "?":
                e=datetime.strptime(date.split(" to ")[1], '%b %d, %Y' )
            else:
                e=pd.to_datetime(np.NaN, errors='coerce')
            end.append(e)
        except:
            end.append('NA')


        #NUM_MEMBER
        try:
            num_mem = anime.find(text = 'Members:').next_element
            nm=int(num_mem.replace('n', '').replace(',', '').strip())
            numMembers.append(nm)
        except:
            numMembers.append(0)


        #SCORE
        try:
            s=anime.find(text = 'Score:').find_next('span').contents
            s=float(s[0])
            score.append(s)
        except:
            score.append(None)


        #USERS
        try:
            us = anime.find(text = 'Score:').find_next('span').find_next('span').contents
            u=int(us[0])
            users.append(u)
        except:
            users.append(0)


        #RANK
        try:
            rk = anime.find(text = 'Ranked:').next_element
            rk=int(rk.replace('\n', '').replace('#', '').strip())
            rank.append(rk)
        except:
            rank.append(None)


        #POPULARITY
        try:
            pop = anime.find(text='Popularity:').next_element
            pop=int(pop.replace("\n","").replace('#', '').strip())
            popularity.append(pop)
        except:
            popularity.append(None)


        #DESCRIPTION
        try:
            des=anime.find(text = 'Synopsis:').find_next('p').text
            des=des.replace("\n","")
            description.append(des)
        except:
            description.append('NA')


        #RELATED
        list = anime.find(text = 'Related Anime')
        rel = []

        if(list != None):

            try:    
                list = list.find_next('list')
                list = list.find_all('a')

                for t in list:
                    rel.append(t.text)

            except:
                for t in list:
                    rel.append('NA')

        related.append(rel)

        #CHARACTER
        c = []
        try:
            list = anime.find(text = 'Characters & Voice Actors').find_next('div')
            list = list.find_all('list')

            for t in list:
                people = t.find_all('h3')
                for p in people:
                    c.append(p.text)
        except:
            c.append('NA')
            
        characters.append(c)


        #VOICES
        v=[]
        try:
            list = anime.find(text = 'Characters & Voice Actors').find_next('div')
            list = list.find_all('h3')

            for t in list:
                people = el.find_next('list')
                for person in people:
                    v.append(person.find('a').text)
        except:
            v.append('NA')

        voices.append(v)

        #STAFF
        st = []
        try:
            list = anime.find_all(text = 'Staff')[1].find_next("div", {"class": "detail-characters-list clearfix"})
            
            if(list != None):    
                table = list.find_all("table")
                for t in table:
                    i = t.find_all("td")[1]
                    p = [i.find("a").text, i.find("small").text]
                    st.append(p)
        except:
            st.append(p)

        staff.append(st)

In [ ]:
#EXECUTE THIS CELL JUST ONES#

col = ['animeTitle', 'animeType', 'animeNumEpisode','releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 
       'animeRank', 'animePopularity', 'animdeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff']

types = {'animeTitle' : 'object', 
         'animeType' : 'object', 
         'animeNumEpisode' : 'int64',
         'releaseDate' : 'datetime64', 
         'endDate' : 'datetime64', 
         'animeNumMembers' : 'int64', 
         'animeScore' : 'float64',
         'animeUsers' : 'int64', 
         'animeRank' : 'int64',
         'animePopularity' : 'int64',
         'animdeDescription' : 'object',
         'animeRelated' : 'object',
         'animeCharacters' : 'object',
         'animeVoices' : 'object',
         'animeStaff' : 'object'}

data = list(zip(title,types,numEpisode,release,end,numMembers,score,users,rank,popularity,description,related,characters,voices,staff))

dataset = pd.DataFrame(data, columns = col).astype(dtype = types)  

for i in range(len(dataset)):
    with open('C:/Users/marti/Desktop/HW3/tsv/anime_'+str(i)+'.tsv', 'w') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow([x for x in dataset.columns]) 
        tsv_writer.writerow(x for x in dataset.iloc[i]) 



# 2. Search engine

In [6]:
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math

# Get a list of the animes
animes = os.listdir('C:/Users/marti/Desktop/tsv_files_full')

# Save column names
names = ['Title', 'Type', 'Episodes', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff', 'release', 'end']

### 2.0.1 Remove stopwords

In [7]:
nltk.download('stopwords') #library for removing stopwords in a text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
def removeStopwords(anime):
    
    # Save the English stopwords in a variable
    en_stops = set(stopwords.words('english'))
    
    for column in anime:
        processed_prop = ''
        for prop in anime[column]:
            if type(prop) is str:
                for word in prop.split():
                    if word not in en_stops: #if the word isn't a stopword, it will be stored in the "processed_prop" variable
                        processed_prop += word + ' '
                processed_df.at[0, column] = processed_prop
            else:
                processed_df.at[0, column] = prop
                
    return processed_df

### 2.0.2 Remove punctuation

In [9]:
import nltk
nltk.download('punkt') #library to remove the punctuation in a text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
def removePunctuation(anime):
    
    for column in anime:
        for prop in anime[column]:
            words = nltk.word_tokenize(str(prop))
            processed = [word for word in words if word.isalnum()]  #if the word isn't a punctuation it will be stored in "proccessed" variable
            processed_df.at[0, column] = ' '.join(processed)
            
    return processed_df

### 2.0.3 Stemming

In [11]:
def stemming(anime):

    ps = PorterStemmer()

    for column in anime:
        processed_prop = ''
        for prop in anime[column]:
            if type(prop) is str:
                for word in prop.split():
                    #for each word, we apply the Porter's Stemmers algorithm, in which we truncate each word to extract the radix
                    processed_prop += ps.stem(word) + ' '       
                processed_df.at[0, column] = processed_prop
            else:
                processed_df.at[0, column] = prop

    return processed_df

In [12]:
for anime_i in range(len(animes)):
    
    anime = pd.read_csv('C:/Users/marti/Desktop/tsv_files_full/anime_{i}.tsv'.format(i=anime_i), sep='\t')

    processed_df = pd.DataFrame(columns=names)    
    processed_df = removeStopwords(anime)
    processed_df = removePunctuation(processed_df)
    processed_df = stemming(processed_df)

    processed_df.to_csv('C:/Users/marti/Desktop/tsv_processed/anime_{i}.tsv'.format(i=anime_i), sep='\t')

### 2.1.1 Create the index

In [13]:
#create a dataframe with the original .tsv files 
animes_lst = []
for anime_i in range(len(animes)):
    anime = pd.read_csv('C:/Users/marti/Desktop/tsv_processed/anime_{i}.tsv'.format(i=anime_i), sep='\t')
    animes_lst.append(anime)

In [14]:
animes_df = pd.concat(animes_lst)
animes_df.drop('Unnamed: 0', axis=1, inplace=True)
animes_df.reset_index(inplace = True, drop=True)
print(animes_df.head())

                              Title Type  Episodes  Members Score    Users  \
0    fullmet alchemist brotherhood   tv         64  2675751   NaN  1622384   
1                               NaN  tv         51   483807   NaN   169476   
2  shingeki kyojin season 3 part 2   tv         10  1596039   NaN  1087519   
3                       stein gate   tv         24  2090910   NaN  1109700   
4           fruit basket the final   tv         13   275214   NaN   113310   

  Rank  Popularity                                        Description  \
0    1           3  after horrif alchemi experi goe wrong elric ho...   
1    2         337  gintoki shinpachi kagura return broke member y...   
2    3          33  restor diminish hope survey corp embark missio...   
3    4          11  mad scientist rintar okab rent room ricketi ol...   
4    5         651  year ago chines zodiac spirit god swore stay t...   

                                             Related  \
0  fullmet alchemist version fullmet

In [15]:
#Create a dictionary in which for each key = word and for each values = term_id, a univoque index for refearing at the word
def create_vocabulary(dataframe):
    vocabulary = {}
    term_id = 1

    for index, anime in dataframe.iterrows():

        description = anime['Description']  
        if type(description) is str:
            words = description.split()  

            for word in words:

                # Create vocabulary
                if word not in vocabulary:
                    vocabulary[word] = term_id
                    term_id += 1
          
    return vocabulary


In [16]:
#Create an inverted index, in which for all the term_id of all the word we store all the document in which appear the word
def create_inverted_index(vocabulary, dataframe):
    inverted_index={}
    for index, anime in dataframe.iterrows():

        description = anime['Description']  
        if type(description) is str:
            words = description.split()  

            for word in words:
                term_id = vocabulary[word]
                if term_id not in inverted_index:
                    inverted_index[term_id] = []
                if index not in inverted_index[term_id]:
                    inverted_index[term_id].append(index)
    return inverted_index

In [17]:
vocabulary=create_vocabulary(animes_df)
inverted_index=create_inverted_index(vocabulary, animes_df)

## 2.1.2 execute query

In [18]:
#necessary to return the original description of the anime (not the processed one)
animes_original = []
for anime_i in range(len(animes)):
    anime_original = pd.read_csv('C:/Users/marti/Desktop/tsv_files_full/anime_{i}.tsv'.format(i=anime_i), sep='\t')
    animes_original.append(anime_original)
animes_original = pd.concat(animes_original)
animes_original = animes_original.reset_index()

In [19]:
import csv
import numpy as np
animes_original.to_csv('C:/Users/marti/Desktop/file_originali_tsv/anime_original.csv')

In [20]:
#necessary to get the urls for each anime
with open('C:/Users/marti/Desktop/HW3/list_anime.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [21]:
#the goal of this function is to return all the document in which appears all the term passed in the string "query"
def execute_query(query):
    term_id=0
    first=True
    query=query.split()
    q_match=pd.DataFrame(columns=['Title', 'Description', 'URL'])

    try:
        for term in query:
            if term in vocabulary:
                term_id=vocabulary[term]
            if first:
                lst_doc=inverted_index[term_id]
                first=False
            else:
                lst_doc=set(lst_doc).intersection(inverted_index[term_id])
                if len(lst_doc)==0:
                    print('no doc found')
                    break

        for i, doc in enumerate(lst_doc):
            q_match.at[i, ['Title', 'Description', 'URL']]=animes_original.iloc[doc]['Title'], animes_original.iloc[doc]['Description'], lines[doc]
        
        display(q_match)
    except:
        print("no doc found")


        

In [22]:
#to execute correctly the execute_query() function we have to preprocess the string given in input
def pre_process_query(query):
    tokenized_query = nltk.word_tokenize(query)
    en_stops = set(stopwords.words('english'))
    ps = PorterStemmer()

    # Process the query before executing it
    processed_final=" "

    remove_punctuation = [t for t in tokenized_query if t.isalnum()]
    
    remove_stopwords = [t for t in remove_punctuation if t not in en_stops]
    
    
    for word in remove_stopwords:
        processed_final +=ps.stem(word)+" "

    
    
    return processed_final


In [23]:
query="home lives house computing texting keyboard phone"
proc=pre_process_query(query)
print(proc)

 home live hous comput text keyboard phone 


## 2.2 Conjunctive query & Ranking score

### 2.2.1) inverted index

In [24]:
#in this inverted_index we add for each document the tf_idf value, an index to to measure the importance of a term with respect to a document or collection of documents
#so for each term_id we have the doc number and the tf_idf -> term_id : (doc, tf_idf)
def create_second_inverted_index(vocabulary, inverted_index, dataframe):

    second_inverted_index={}
    
    for index, anime in dataframe.iterrows():
        vocabulary_occurrences = {}
        occurrence=[]

        description = anime['Description']  
        if type(description) is str:
            words = description.split()  

            for word in words:
                if word not in occurrence:
                    vocabulary_occurrences[word]=words.count(word)


            tot_words_in_doc=len(words)

            for word in vocabulary_occurrences:
                term_id=vocabulary[word]

                #term frequency=#word appears in doc / #tot words in doc
                word_occur=vocabulary_occurrences[word]
                tf=word_occur/tot_words_in_doc

                #inverse data frequency= log(#tot_docs/#docs_containing_term_i)
                tot_docs=len(dataframe)
                docs_with_word=len(inverted_index[term_id])
                idf=math.log(tot_docs/docs_with_word, 10)

                #tf-idf
                tf_idf=tf*idf

                if term_id not in second_inverted_index:
                    second_inverted_index[term_id] = []
                if index not in second_inverted_index[term_id]:
                    second_inverted_index[term_id].append((index, tf_idf))

    return second_inverted_index

    


In [25]:
vocabulary = create_vocabulary(animes_df)
inverted_index = create_inverted_index(vocabulary, animes_df)

second_inverted_index=create_second_inverted_index(vocabulary, inverted_index, animes_df)

### 2.2.2) Execute query

In [26]:
#this function convert all the document in a list of his term_id
# doc : term_id  for all the term_id in doc
def documents_to_word(dataframe, doc, vocabulary):
    vocabulary_occurrences={}
    occurrence=[]
    description=dataframe.iloc[doc]['Description']
    norm1=0
    description=description.split()

    
    for word in description:

        if word not in occurrence:
            vocabulary_occurrences[word]=description.count(word)



    tot_words_in_doc=len(description)

    for word in vocabulary_occurrences:
        term_id=vocabulary[word]

        #term frequency=#word appears in doc / #tot words in doc
        word_occur=vocabulary_occurrences[word]
        tf=word_occur/tot_words_in_doc

        #inverse data frequency= log(#tot_docs/#docs_containing_term_i)
        tot_docs=len(dataframe)
        docs_with_word=len(inverted_index[term_id])
        idf=math.log(tot_docs/docs_with_word, 10)

        #tf-idf
        tf_idf=tf*idf        
        
        norm1 += np.power(tf_idf, 2)

    return norm1
      

In [27]:
dict_documents=documents_to_word(animes_df, 1, vocabulary) 
print(dict_documents)  

0.07765361421590745


In [28]:
#this function calculate the cosine similarity between the query and the description in all the documents
def cosine_similarity(query):

    query=pre_process_query(query)
    cos_similarities = []
    ti_query={}
    q_match=pd.DataFrame(columns=['Title', 'Description', 'Similarity'])
    query=query.split()
    try:
        for word in query:
            tf=query.count(word)/len(query)
            term_id=vocabulary[word]
            idf=np.log(len(second_inverted_index)/len(second_inverted_index[term_id]))
            ti_query[word]=tf*idf


        lst_doc = []
        for term in query:
            if term in vocabulary:
                term_id = vocabulary[term]
                lst_doc.append(inverted_index[term_id])
            else:
                print("no doc found")
                return

        set_doc=lst_doc[0]
        for x in lst_doc:
            set_doc=set(set_doc).intersection(set(x))


        set_doc=sorted(set_doc)
        set_doc=list(set_doc)

        #denominator 2
        norm2 = 0

        for q in query:
            i = ti_query[q]
            norm2 += np.power(i, 2)
        norm2 = np.sqrt(norm2)


        for term in query:
            term_id=vocabulary[term]
        
            
            #scalar product between tf_idf * #number of occurrences of the term in the query 
            for i, doc in enumerate(set_doc):
                c = 0
                tf_idf=second_inverted_index[term_id][i][1]
                c +=  tf_idf * ti_query[term]

                #denominator 1
                
                norm1=documents_to_word(animes_df, doc, vocabulary) 


                norm1 = np.sqrt(norm1)


                tot=c / (norm1 * norm2)
                cos_similarities.append(tot)

        

        for i, doc in enumerate(set_doc):
            q_match.at[i, ['Doc', 'Title', 'Description', 'URL', 'Similarity']] = doc, animes_original.iloc[doc]['Title'], animes_original.iloc[doc]['Description'], lines[doc], cos_similarities[i]
            q_match.sort_values(by=['Similarity'], ascending=False, inplace=True)
            q_match.reset_index(inplace=True, drop=True)
        
        return q_match
    except:
        print("no doc found")

In [29]:
cosine_similarity("saiyan race")

,Title,Description,Similarity,Doc,URL
0,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"[""Bardock, Son Goku's father, is a low-ranking...",0.24986,1467.0,https://myanimelist.net/anime/986/Dragon_Ball_...
1,Dragon Ball Super: Broly,"[""Forty-one years ago on Planet Vegeta, home o...",0.15304,400.0,https://myanimelist.net/anime/1364/Detective_C...
2,Dragon Ball Z,"[""Five years after winning the World Martial A...",0.097484,364.0,https://myanimelist.net/anime/813/Dragon_Ball_Z\n
3,Dragon Ball Kai,"[""Five years after the events of Dragon Ball, ...",0.079373,1035.0,https://myanimelist.net/anime/31483/Akagami_no...


## 3. Define a new score

#### Custom score = cos.similarity * 0.4 + type * 0.4 + episodes * 0.1 + popularity * 0.1

This index is based on fuor different concepts, the score is higher when the query is more similar to the document, the score increases when the type of the anime match with respect to the type chosen by the users, other important point is that to many people don't like series that are too long, so it's important to increase the score when the number of episodes chosen by the users and those of the dataset and at the end we all know how important the fashions are therefore we want our index to be higher if user's popularity match with our popukarity's range.

In [30]:
import heapq as hq


In [31]:
#calculate the new_score as described above
def new_score(dataframe, query, anime_type, episode_min, episode_max, pop_min, pop_max, vocabulary):
    df_cos=cosine_similarity(query)
    query=pre_process_query(query)
    query=query.split()
    lst_new_score={}
    for index, anime in dataframe.iterrows():
        lst_doc=[]
        for term in query:
            if term in vocabulary:
                term_id = vocabulary[term]
                lst_doc.append(inverted_index[term_id])
            else:
                print("no doc found - new score")
                return

                
    set_doc=lst_doc[0]
    for x in lst_doc:
        set_doc=set(set_doc).intersection(set(x))


    set_doc=sorted(set_doc)
    set_doc=list(set_doc)

    for doc in set_doc:
        row=df_cos.loc[df_cos['Doc']==doc]
        cos_sim=row.iloc[0]["Similarity"]
        cos_sim=float(cos_sim)

        t_anime=0
        n_episode=0
        popularity_anime=0

        if(anime['Type'] and anime['Type']==anime_type): t_anime=1.0
        if(anime['Episodes']>= episode_min and anime['Episodes']<= episode_max): n_episode=1.0 
        if(anime['Popularity']>= pop_min and anime['Popularity']<= pop_max): popularity_anime=1.0

        my_score=cos_sim * 0.4 + t_anime * 0.4 + n_episode * 0.1 + popularity_anime * 0.1
        lst_new_score[doc]=my_score


    heap = [(-value, key) for key,value in lst_new_score.items()]
    largest = hq.nsmallest(10, heap)
    largest = [(key, -value) for value, key in largest]
    
    matching=dict(largest)

    q_match=pd.DataFrame()
    for i, doc in enumerate(matching.keys()):
        q_match.at[i, ['Doc', 'Title', 'Description', 'URL', 'Similarity']] = int(doc), animes_original.iloc[doc]['Title'], animes_original.iloc[doc]['Description'], lines[doc], matching[doc]
        q_match.reset_index(inplace=True, drop=True)

    display(q_match)


In [32]:
#function to insert the query from input
def query_users():

    print("insert a query: ")
    query=input()
    print(query)
    
    print("What type of anime do you want?")
    anime_type=input()
    print(anime_type)

    print("How many episode minimum do you want?")
    episode_min=input()
    print(episode_min)

    print("How many episode maximum do you want?")
    episode_max=input()
    print(episode_max)

    print("What popularity minimum do you want?")
    pop_min=input()
    print(pop_min)

    print("What popularity maximum do you want?")
    pop_max=input()
    print(pop_max)


    vocabulary = create_vocabulary(animes_df)
    n_score=new_score(animes_df, query, anime_type, int(episode_min), int(episode_max), int(pop_min), int(pop_max), vocabulary)

    return n_score

In [33]:
query_users()

insert a query: 
dragon ball
What type of anime do you want?
tv
How many episode minimum do you want?
1
How many episode maximum do you want?
1000
What popularity minimum do you want?
1
What popularity maximum do you want?
2000


,Doc,Title,Description,URL,Similarity
0,5985.0,Dragon Ball GT: Gokuu Gaiden! Yuuki no Akashi ...,"[""Years after the end of the Dragonball GT, th...",https://myanimelist.net/anime/37422/Cinderella...,0.143356
1,364.0,Dragon Ball Z,"[""Five years after winning the World Martial A...",https://myanimelist.net/anime/813/Dragon_Ball_Z\n,0.136394
2,6557.0,Galo Sengen,"['Galo Sengen (""Galish Man GAL"" in English / G...",https://myanimelist.net/anime/40526/Dragon_Ie_...,0.097717
3,2267.0,Dragon Ball Z Movie 13: Ryuuken Bakuhatsu!! Go...,['The Z Warriors discover an unopenable music ...,https://myanimelist.net/anime/26349/Danna_ga_N...,0.046869
4,5275.0,Dragon Ball Z Movie 01: Ora no Gohan wo Kaese!!,['Piccolo is training at a barren cliff when a...,https://myanimelist.net/anime/2814/Dondon_Domm...,0.046508
5,6595.0,Nezha Nao Hai,['The film is an adaptation of a story in Chin...,https://myanimelist.net/anime/14693/Yurumates_...,0.044271
6,11479.0,Kyutai Panic Adventure Returns!,['A special screened at an event named Odaiba ...,https://myanimelist.net/anime/30664/Jinkou_no_...,0.042163
7,3224.0,Dragon Ball Movie 4: Saikyou e no Michi,"[""A retelling of Dragon Ball's origin with a d...",https://myanimelist.net/anime/2664/Doraemon_Mo...,0.042137
8,6340.0,Dragon Ball Z: Atsumare! Gokuu World,"[""Dragon Ball Z: Atsumare! Goku's World is a T...",https://myanimelist.net/anime/1153/Crying_Free...,0.038425
9,5760.0,Dragon Ball Z Movie 04: Super Saiyajin da Son ...,"[""Gohan Son and Piccolo are peacefully playing...",https://myanimelist.net/anime/9549/Alps_no_Sho...,0.036603


## 5. Algorithmic question

In the given problem, the goal is to choose the maximum number of appointments such as the appointments choosen are not consecutive. It means that the personal trainer want needs a break between appointments and so he can't accept two consecutive request. Obviously, all the request are consider in chronological order. The input data is just a list of the requested appointment, and we want to maximize the sum of value (not adiacent) contained in the list.

One solution could be the followed:


In [38]:
def find_max_sum(arr):
    a = 0
    b = 0
    lst=[]
    for i in arr:

        if b>a:
            c= b
        else:
            c=a
         
        a = b + i
        b = c
     
    # return max of a and b
    return (b if b>a else a)

In [39]:
app = [30, 40, 25, 10, 20, 50, 30, 20] 
print('Appointments required:', app, '\n') 

duration=find_max_sum(app)

print('total duration (of the accepted appointment): ', duration)

Appointments required: [30, 40, 25, 10, 20, 50, 30, 20] 

total duration (of the accepted appointment):  125


In this algorithm we do not save the element that compose the final sum. For this reason we have to modify this algorithm to store and provide the sub set of the initial array, as follow:

In [40]:
def find_max_subs(appointment):
    
    sums = [0 for j in range(len(appointment))] #appointment for store all sums
    
    accepted = dict() #we will save in a dictionary all the accepted appointment
    
    for i in range(len(appointment)):

        sums[i] = max(sums[i-1], sums[i-2] + appointment[i]) #choose the sums between the last sum and the updated one with the i-th element
        
        if max(sums[i-1], sums[i-2] + appointment[i]) == (sums[i-2] + appointment[i]): #store the sum
            accepted[sums[i-2], appointment[i]] = sums[i]
            
    
    # we want also to know the element that compose the sums, in the follow list we will store the optimal solution 
    list_of_success = []
    max_value = max(accepted.values())
    
    #a flag is needed to terminate the while when the sum_values is equal to zero (and so when we finished the l)
    flag = list(accepted.keys())[list(accepted.values()).index(max_value)][0]

    while flag != 0:
        
        #having an element of the dictionary as (x, y) : z, we store x in single_values and y in sum_value         
        single_values = list(accepted.keys())[list(accepted.values()).index(max_value)][1]  #variable in which we store the single value
        sum_value = list(accepted.keys())[list(accepted.values()).index(max_value)][0]      #variable in which we store the cumulative number
        
        #the single_value is exactly the one choosen by the algorithm, so we store it
        list_of_success.append(single_values)
        
        #update the executing variable
        max_value = sum_value
        flag = sum_value
        
    #since we operate from the bottom to the up of the solution, we have to reverse the final array
    list_of_success.reverse()
    
    return sums[-1], list_of_success

In [41]:
app = [30, 40, 25, 50, 30, 20] 
print('Appointments required:', app, '\n') 

duration, subs=find_max_subs(app)

print('total duration (of the accepted appointment): ', duration, " with the sub set of elements: ", subs)

Appointments required: [30, 40, 25, 50, 30, 20] 

total duration (of the accepted appointment):  110  with the sub set of elements:  [40, 50, 20]
